# PowerPlant Dataset : Arnaud PIGNEROL & Louis TARDY

## Installation of libraries

In [14]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression

## Obtention of the excel file

In [15]:
df = pd.read_excel("..\CCPP\Folds5x2_pp.xlsx", sheet_name = None)
df = pd.concat(df, axis = 0, ignore_index = True)

## We have to normalize our data

In [16]:
def normalizeFeature(df):
    normalized = (df - df.min()) / (df.max() - df.min())
    df.update(normalized)


normalizeFeature(df)
df.head()

,AT,V,AP,RH,PE
0,0.372521,0.291815,0.771591,0.638204,0.569536
1,0.662040,0.669039,0.671863,0.449330,0.319338
2,0.093484,0.249822,0.476862,0.892493,0.904636
3,0.539660,0.568683,0.429349,0.684718,0.347285
4,0.255241,0.216014,0.404355,0.952547,0.710464


## Now we can extract our values

In [17]:
x = df[['AT', 'V', 'AP', 'RH']]
y = df[['PE']]

n = len(x.axes[1]) # n is the number of observations = 4
m = len(x.axes[0]) # m is the number of features = 47840

X = np.c_[np.ones((m, 1), dtype = float), x]

## We have to implement the vector of weights w
### Because we do not have weights yet we instantiate it at 1

In [18]:
w = np.ones((n + 1, 1))
w

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

## We have to implement the cost function J
$ J_w =\frac {1}{2m} * \sum (h_w(x^i) - y^i)² $

In [19]:
Cost = []
def costFunction(w, X, y):    
    hw = np.dot(X, w)
    val = hw - y
    val2 = val ** 2
    somme = np.sum(val2)
    
    result = somme / (2 * len(y))
    return result[0]

J = costFunction(w, X, y)
Cost.append(costFunction(w, X, y))
Cost

[3.815852728463007]

## We have to implement the Gradient Descent
$ w_k = w_k - \frac {\alpha}{m} * \sum (h_w(x^i) - y^i) * x_k $

In [26]:
gradient = np.zeros((n + 1, 1))
def gradientDescent(w, X, y):    
    
    max_iterations = 1000
    delta = 0.000000001
    alpha = 0.03
    
    for i in range(max_iterations):
        tmp = w
        
        hw = np.dot(X, w)
        val = hw - y
        gradient = np.dot(X.transpose(), val)
        
        w = tmp - gradient * alpha
        
        errN = costFunction(w, X, y)
        errPrev = Cost[-1]
        Cost.append(errN)
        
        if(np.abs(errN - errPrev) < delta):
            print("Done\n", errN, "\n", errPrev)
            break
            
        if(errN > errPrev):
            alpha = alpha / n
            print("alpha reduced : ", alpha)
        
    return w


w = gradientDescent(w, X, y)
w

alpha reduced :  0.0075
alpha reduced :  0.001875
alpha reduced :  0.00046875
alpha reduced :  0.0001171875
alpha reduced :  2.9296875e-05
alpha reduced :  7.32421875e-06


array([[ 1.70497643],
       [-1.6595034 ],
       [ 0.19364774],
       [-0.39622948],
       [-0.48742235]])

## Once you build a good model with your from-scratch implementation, build a new model with Scikit-Learn and compare the obtained results.
### Comparaison to the result obtain with Scikit-Learn

In [27]:
model = LinearRegression()
model.fit(X, y)
model.coef_
model.intercept_
model.coef_[0][0] = model.intercept_

print("theta coef : {0}".format(model.coef_))
print("w           ", w.transpose())

theta coef : [[ 1.09191426 -0.9245856  -0.17412057  0.03322877 -0.15617001]]
w            [[ 1.70497643 -1.6595034   0.19364774 -0.39622948 -0.48742235]]


## Add the following functions to your library to calculate R2, MAE, RMSE
### 1. meanAbsoluteError()
$ mae = \frac {1}{N} * \sum |y_i - \bar{y}| $

In [28]:
# MAE
def meanAbsoluteError(predicted, y):
    return ((np.abs(y - predicted)).sum() / len(y))[0]



mae = meanAbsoluteError(np.dot(X, w), y)
print(mae)

0.07362888305826783


### 2. rootMeanSquaredError()
$ rmse = \sqrt{ \frac {1}{N} * \sum (y_i - \hat{y})²} $

In [29]:
# RMSE
def rootMeanSquaredError(predicted, y):
    return (math.sqrt(np.sum((y - predicted) ** 2) / len(y)))


rmse = rootMeanSquaredError(np.dot(X, w), y)
print(rmse)

0.09331990200063878


### 3. r2()
$ r² = 1 - \frac {\sum (y_i - \hat{y})²} {\sum (y_i - \bar{y})²} $

In [30]:
# R2
def R2(predicted, y):
    
    # numerator
    error = (y - predicted) ** 2
    error2 = np.sum(error)[0]
    
    # denomirator
    mean = y.mean()
    variance = (y - mean) ** 2
    variance2 = np.sum(variance)[0]
    
    return (1 - (error2 / variance2))



r2 = R2(np.dot(X, w), y)
print(r2)

0.8295591381673992


## Use LinearRegression from sklearn.linear_model to build a new linear model
### Compare the obtained R2

In [31]:
clf = LinearRegression()
clf.fit(np.dot(X, w), y)

print(clf.score(np.dot(X, w), y))
print(r2)

0.8552083951559957
0.8295591381673992


## Write a function to calculate the optimal values of wi using the normal equation

#### normal_equation = $ \theta_0 + \theta_1 * y_1 + ... $
#### using $ \theta = inv(X^T * X) * (X^T * y) $

In [34]:
def normalEquation(X, y):
    return np.dot(np.linalg.inv(np.dot(X.transpose(), X)), np.dot(X.transpose(), y))

normalEq = normalEquation(X, y)
print(normalEq)

[[ 1.09191426]
 [-0.9245856 ]
 [-0.17412057]
 [ 0.03322877]
 [-0.15617001]]


## Compare w from the normal equation and from the gradient

In [42]:
print("w       ", w.transpose())
print("normalEq", normalEq.transpose())

w        [[ 1.06234424 -0.88396279 -0.19690041  0.05173624 -0.13846393]]
normalEq [[ 1.09191426 -0.9245856  -0.17412057  0.03322877 -0.15617001]]
